In [22]:
from typing import Any, Dict, Optional, Tuple, List
import torch
from pytorch_lightning import LightningDataModule
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from typing import Union
import os
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np

### preprocess wellness dataset

In [4]:
path = "../data/chatbot/Wellness Conversation Script Dataset/wellness.xlsx"
df = pd.read_excel(path)
df.drop('구분', axis=1, inplace=True)
df.rename(columns={'유저': 'text','챗봇': 'answer'}, inplace=True)
df.dropna(axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,text,answer
0,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
3,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.
4,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.


In [5]:
df.to_csv("../data/chatbot/Wellness Conversation Script Dataset/wellness.csv", index=False)

In [6]:
df2 = pd.read_csv("../data/chatbot/Wellness Conversation Script Dataset/wellness.csv", encoding='utf-8')
df2.head()

,text,answer
0,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
3,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.
4,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.


In [70]:
path = '../logs/train/runs/2022-11-02_15-05-53/chatbot_db/chatbot_db.parquet'

df = pd.read_parquet(path)
print(df.dtypes)
df.head()

answer       object
embedding    object
dtype: object


,answer,embedding
0,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.51023406, -0.3484826, 0.48231104, -0.65767..."
1,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1405175, -0.15230586, 0.39684805, -0.64085..."
2,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.7149122, -0.09625367, 1.1156603, -0.057317..."
3,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.690742, 0.35730758, 0.47781637, -0.4919525..."
4,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.15590465, -0.35547465, 0.040454123, -0.951..."


In [34]:
import pandas as pd 
import time
import torch

def convert_size(size_bytes):
    import math
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])

file = '../logs/train/runs/2022-11-02_15-05-53/chatbot_db/chatbot_db.parquet'
df = pd.read_parquet(file)

print("[CSV]")
start = time.time()
df.to_csv("./file.csv", index =False) 
print("Write csv time :", time.time() - start)

start = time.time()
df = pd.read_csv("./file.csv")
print("Read csv time :", time.time() - start)

file_size = os.path.getsize('./file.csv') 
print('csv File Size:', convert_size(file_size))

[CSV]
Write csv time : 1.9682483673095703
Read csv time : 0.06273055076599121
csv File Size: 12.09 MB


In [32]:
import pickle

print("[Pickle]")
start = time.time()
df.to_pickle("./file.pkl") 
print("Write pickle time :", time.time() - start)

start = time.time()
df = pd.read_pickle("./file.pkl")
print("Read pickle time :", time.time() - start)

file_size = os.path.getsize('./file.pkl') 
print('csv File Size:', convert_size(file_size))

[Pickle]
Write pickle time : 0.010529041290283203
Read pickle time : 0.007519721984863281
csv File Size: 12.09 MB


In [39]:
import pandas as pd 
import time

print("[Parquet(Snappy)]")
start = time.time()
df.to_parquet("./file.parquet", compression="snappy")
print("Write parquet time :", time.time() - start)
 
start = time.time()
df = pd.read_parquet("./file.parquet")
print("Read parquet time :", time.time() - start)

file_size = os.path.getsize('./file.parquet') 
print('parquet File Size:', convert_size(file_size))


print("\n[Parquet(gzip)]")
start = time.time()
df.to_parquet("./file.parquet", compression="gzip")
print("Write parquet time :", time.time() - start)
 
start = time.time()
df = pd.read_parquet("./file.parquet")
print("Read parquet time :", time.time() - start)

file_size = os.path.getsize('./file.parquet') 
print('parquet File Size:', convert_size(file_size))

[Parquet(Snappy)]
Write parquet time : 0.06501150131225586
Read parquet time : 0.04352164268493652
parquet File Size: 7.89 MB

[Parquet(gzip)]
Write parquet time : 0.584850549697876
Read parquet time : 0.05291938781738281
parquet File Size: 4.66 MB
